# Tensors


**Тенхзор - это структура данных, хранящая набор чисел, к которым можно обращаться по отдельности с помощью индекса, причем индексов может быть несколько.**

In [1]:
import random
import warnings

import numpy as np
import torch


RND = 21
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# deterministic
random.seed(RND)
np.random.seed(RND)
torch.manual_seed(RND)
torch.cuda.manual_seed(RND)
torch.backends.cudnn.deterministic = True

# syntactic sugar
warnings.filterwarnings("ignore")

## 1.1 Pyton list VS Tensor

In [2]:
a = [1.0, 2.0, 3.0]

К элементу этого списка можно обратиться по соответсвующим индексам, начиная с нулевого.

In [3]:
a[0]

1.0

In [4]:
a[2]

3.0

First tensor

In [5]:
a = torch.ones(3)

In [6]:
a

tensor([1., 1., 1.])

In [7]:
a[1]

tensor(1.)

In [8]:
float(a[1])

1.0

In [9]:
a[2] = 3

a

tensor([1., 1., 3.])

**Хотя внешне этот пример не слишком отличается от списка числовых объектов "за кулисам" все сильно отличается**

### Что такое тензор?

Списки и значения числовых значений Python представляют собой наборы объектов python, память под которые выделяется по отдельности.

**Тензоры и массивы Numpy являются представлениями над (обычно) непрерывными блоками памяти, содержащими распакованные (unboxed) числовые типы данных С, а не объекты Python.**

В нашем случае каждый элемент представляет собой 32-битное (4 байта) значения типа **float**. 

А это значит что для хранения 1 000 000 чисел типа float нам потребуется непрерывная область памяти в 4 000 000 байт + небольшое дополнительное место для метаданных. (размерности и числовой тип)

### Доступ к тензорам по индексам

Нотация диапазонного доступа по индексу в Python:

In [10]:
a = [(i) for i in range(10)]

In [11]:
print(a)
print(a[::-1])
print(a[2:-1])
print(a[::2])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
[2, 3, 4, 5, 6, 7, 8]
[0, 2, 4, 6, 8]


В Pytorch можно использовать ту же нотацию типов **с дополнительным плюсом** в виде возможности использовать в виде возможности использовать диапазонный доступ по индексу для каждого из измерений тензора:

In [12]:
del a

In [13]:
a = torch.tensor([[1., 2.], [3., 4.], [5., 6.]])

In [14]:
print(a[1:])
print(a[1:, :])
print(a[1:, 1:])
print(a[1:, 0])
print(a[None]) # unsqueeze

tensor([[3., 4.],
        [5., 6.]])
tensor([[3., 4.],
        [5., 6.]])
tensor([[4.],
        [6.]])
tensor([3., 5.])
tensor([[[1., 2.],
         [3., 4.],
         [5., 6.]]])


### Поименованные тензоры

Измерения (оси координат) тензоров обычно соответсвуют чему-то на подобие расположения пикселей или цветных канало. А это значит, что при доступе к тензору по индексам необходимо помнить порядок измерений и записывать индексы соответсвующим образом. Отслеживание того какое измерение содержит какие данные, может приводить к возникновению ошибок при преобразовании двнных через несколько тензоров.

**Пример:**

Дан трехмерный тензор, который необходимо преобразовать в оттенки серого. Ищем типичные весовые коэффициенты цветов для получения одного единого значения яркости:

In [15]:
img_t = torch.rand(3, 5, 5) # форма [каналы, строки, столбцы]
weights = torch.tensor([0.2116, 0.7152, 0.0722])

In [16]:
batch_t = torch.rand(2, 3, 5, 5) # форма [батч, каналы, строки, столбцы]
batch_t

tensor([[[[0.1752, 0.3951, 0.2414, 0.1056, 0.0685],
          [0.0250, 0.2968, 0.0507, 0.9776, 0.4003],
          [0.4270, 0.6945, 0.7185, 0.9751, 0.1906],
          [0.4482, 0.5732, 0.0494, 0.4219, 0.8757],
          [0.3706, 0.8420, 0.0281, 0.2398, 0.3721]],

         [[0.9084, 0.0616, 0.3239, 0.4604, 0.3143],
          [0.9487, 0.9416, 0.9318, 0.7542, 0.6262],
          [0.7319, 0.7910, 0.1904, 0.8096, 0.1572],
          [0.7041, 0.8846, 0.5671, 0.9018, 0.8865],
          [0.9801, 0.7626, 0.1479, 0.9386, 0.4176]],

         [[0.7609, 0.4928, 0.7538, 0.7552, 0.3135],
          [0.7059, 0.3626, 0.2118, 0.6789, 0.9255],
          [0.7055, 0.2060, 0.8446, 0.7541, 0.6133],
          [0.0016, 0.5020, 0.8871, 0.9675, 0.5757],
          [0.6287, 0.4687, 0.4433, 0.5450, 0.1721]]],


        [[[0.5491, 0.3465, 0.3457, 0.4364, 0.8697],
          [0.6266, 0.7121, 0.1769, 0.8595, 0.3286],
          [0.4796, 0.2790, 0.2428, 0.3188, 0.6214],
          [0.5538, 0.8634, 0.5635, 0.0851, 0.4550],
    

Иногда каналы размещаются в измерении 0, а иногда - в измерении 1.
Но обобщение можно производить путем отсчета с конца: они всегда расположены в -3 с конца.

Вычисление невзвешенного среднего можно записать следующим образом:

In [17]:
img_gray_navie = img_t.mean(-3)
batch_gray_navie = batch_t.mean(-3)

img_gray_navie.shape, batch_gray_navie.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

### Работа с атрибутом dtype тензоров

In [18]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [19]:
print(double_points, '\n')
print(short_points)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], dtype=torch.float64) 

tensor([[1, 2],
        [3, 4]], dtype=torch.int16)


Получить информацию о dtype тензора можно путем обращения к ссответствующему атрибуту:

In [20]:
short_points.dtype

torch.int16

Можно также привести результат ф-ции создания тензора к нужному типу с помощью соответствующего метода приведения типов

In [21]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

Или более удобного метода to:

In [22]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(torch.short)

**"За кулисами"** to проверяет необходимость преобразования и производит его, если нужно.

Методы приведения типов dtype например float, являются сокращенными формамим вызова метода to, но они могут также принимать дополнительные аргументы.

**ПРИ СМЕШИВАНИИ ВХОДНЫХ ТИПОВ ДАННЫХ В ОПЕРАЦИЯХ ДАННЫЕ АВТОМАТИЧЕСКИ ПРЕОБРАЗУЮТСЯ К БОЛЬШЕМУ ТИПУ.** Следовательно, если нам нужны 32-битные вычисления, нужно убедится, что все входные сигналы (как минимум) 32-битные.

***
### Операции над тензорами

Лучше смотреть в доке: [клик](https://pytorch.org/docs/stable/tensors.html)

***

## Тензоры: Хранение в памяти

Память под значения в тензорах выделяется непрерывными фрагментами под управлением экземпляров **torch.Storage**. Хранилище предстовляет собой одномерный массив числовыхх данных, то есть непрерывный фрагмент в памяти, содержащий числа заданного типа. Экземпляр класса **Tensor Pytorch** - это представление подобного экземпляра **Storage** с возможностью доступа к хранилищу по индексу через указание сдвига и шага по каждому измерению.

**Несколько тензоров могут обращаться к одному хранилищу, даже если индексация данных происходит по разному.**

Лежащая в основе тензора память выделяется только один раз, благодаря чему создание различных тензорных представлений данных происходит очень быстро вне зависимости от размера данных, контролируемых экземпляром **Storage**.

## Доступ к хранилищу по индексу

Обращаться к конкретному хранилищу тензора можно посредством его свойств **.storage**

In [23]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

И хотя тензор сообщает, что содержит три строки и два столбца, лежащее в его основе хранилище представляет собой непрерывный массив размером 6. В этом смысле тензор просто знает, как преобразовать пару индексов в место в этом хранилище.

Мы не сможем обращаться по индексу к хранилищу двумерного тензора с помощью двух индексов. Хранилище всегда представляет собой одномерный массив вне зависимости от размерности каких-либо ссылающихся на него тензоров.

Теперь нас наврятли удивит, что изменение значения в хранилище приводит к изменению содержимого тензора,который на него ссылается:

In [24]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

### Модификация хранимых значений: операции с заменой на месте

Помимо представленных в предыдущем разделе операций над тензорами, небольшое кол-во операций доступно только в виде методов объекта **Tensor**. Их можно отличить по подчеркиванию в конце названия, как в zero_, указывающему, что метод работает с заменой на месте (in place), изменяя входные данные вместо того, чтобы создавать новый выходной тензор и возвращать его. Например, метод zero_ обнуляет все элементы входного тензора. 

Все методы, **в конце названия которых НЕТ символов подчеркивания**, оставляют исходный тензор неизменным и вместо этого возвращают новый.

In [25]:
a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

***
## Метаданные тензоров: размер, сдвиг и шаг

Для доступа к хранилищу по индексу в тензорах используется несколько элементов информации, которые вместе с хранилищем однозначно определяют их: размер, сдвиг, шаг. Размер (форма в NumPy) представляет собой кортеж, содержащий число элементов тензора по каждому измерению. 

Сдвиг хранилища - это индекс в хранилище, соответствующий первому элементу тензора.

Шаг - это кол-во элементов хранилища, пропускаемых между последовательными элементами по каждому измерению.

### Представления хранилища другого тензора

Мы можем получить вторую точку из тензора, указав соответствующий индекс

In [26]:
points = torch.tensor([[[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]], [[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]]])
second_point = points[1]
second_point.storage_offset() # Возвращает смещение элемента тензора и первого элемента хранения.


6

In [27]:
second_point.size()

torch.Size([3, 2])

Сдвиг полученного в результате тензора в хранилище равен 2 (поскольку мы пропускаем первую точку, содержащую два элемента), а размер представляет собой экземпляр класса **Size**, содержащий один элемент, поскольку наш тензор одномерный.

Шаг представляет собой кортеж, указывающий в себе число элементов в хранилище, пропускаемое при увеличении индекса на 1 по каждому измерению. Например, шаг тензора **points** равен (2,1)

In [28]:
points_storage = points.storage()
points

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]],

        [[4., 1.],
         [5., 3.],
         [2., 1.]]])

In [29]:
points_storage

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 12]

In [30]:
points.stride()

(6, 2, 1)

In [31]:
points[0,1]

tensor([5., 3.])

Доступ по индексу производится к тому же хранилищу, что и у исходного тензора **points**. А это значит, что побочным эффектом изменения подтензора станет изменение исходного тензора:

In [32]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

Такое поведение не всегда желательно, так что имеет смысл перезаписать наш подтензор в новый тензор:

In [33]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

## Транспонирование без копирования

Попробуем теперь транспонировать тензор. Возьмем наш тензор **points**, в котором отдельные точки отсчитываются по строкам, а координаты x и y - по столбцам, и транспонируем его, чтобы отдельные точки отсчитывались по столбцам. Воспользуемся этим случаем , чтобы познакомить вас с функцией **t** - сокращенной записью ф-ции **transpose** для двумерных тензоров.

In [34]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [35]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

Можно легко убедится, что **хранилище у этих двух тензоров одно**:

In [36]:
id(points.storage()) == id(points_t.storage())

True

И что они отличаются только формой и шагом:

In [37]:
points.stride()

(2, 1)

In [38]:
points_t.stride()

(1, 2)

In [39]:
points.shape

torch.Size([3, 2])

### Транспонирование при более высокой размерности

Транспонировать в Pytorch можно не только матрицы. Можно транспонировать многомерный массив, и для этого достаточно указать два измерения, по которым нужно произвести транспонирование (зеркально отражая форму и шаг):

In [40]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0,2)
some_t.shape

torch.Size([3, 4, 5])

In [41]:
transpose_t.shape

torch.Size([5, 4, 3])

In [42]:
some_t.stride()

(20, 5, 1)

In [43]:
transpose_t.stride()

(1, 5, 20)

Непрерывные тензоры удобны тем, что их можно эффективно просматривать по порядку, не прыгая по хранилищу от одного элемента к другому (улучшение лояльности данных повышает производительность благодаря тому, как происходит доступ к памяти в современных CPU). Конечно это зависит от способа выполнения алгоритмов обращения.

### Непрерывные тензоры

Непрерывный тензор - это тензор, элементы которого хранятся в непрерывном порядке, не оставляя пустого пространства между ними. Тензор, созданный изначально, всегда является непрерывным тензором. Тензор можно просматривать в разных измерениях непрерывным образом.

**Транспонирование тензора создает представление исходного тензора, которое следует несмежному порядку. Транспонирование тензора не является непрерывным.**

Некоторые тензорные операции наподобие **viev**, в Pytorch работают только для непрерывных тензоров.

В подобном случае PyTorch формирует информативное исключение и требует явного вызова ф-ции **contiguous**. Стоит отметить, что если тензор уже непрерывный, то никаких действий при вызове **contiguous** не производится (и на производительности это никак не сказывается)

In [44]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [45]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

В нашем случае тензор points непрерывный, а транспонированный к нему - нет.

In [46]:
points.is_contiguous()

True

In [47]:
points_t.is_contiguous()

False

Получить новый непрерывный тензор из ненепрерывного можно с помощью метода **contiguous**. Содержимое этого нового тензора будет таким же, но шаг, как хранилище, изменится.

In [48]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [49]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [50]:
points_t.stride()

(1, 2)

In [51]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [52]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [53]:
points_t_cont.stride()

(3, 1)

Новое хранилище было перетасованно, чтобы в новом хранилище элементы располагались построчно. Шаг также был изменен в соответствие с новым размещением элементов.

***

## Перенос тензоров на GPU

Любой из тензоров PyTorch можно перенести на (один из) GPU системы для массово параллельных быстрых вычислений. После этого все производимые с этим тензором операции будут выполнятся с помощью специализированных процедур для работы с GPU, включенных в PyTorch.

### работа с атрибутом device тензоров

Помимо **dtype**, класс Tensor представляет атрибут **device**, который описывает, где на компьютере размещаются данные тензора. Вот как можно создать тензор в GPU, указав в конструкторе соответствующий аргумент.

In [54]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]], device='cuda')

Вместо этого можно скопировать созданный в CPU тензор на GPU с помощью метода **to**:

In [55]:
points_gpu = points.to(device='cuda')

При этом возвращается новый тензор с теми же числовыми данными, но хранящийся в памяти GPU, а не в обычной оперативной памяти системы.


Если на нашей машине больше одного GPU, можно также указать, на каком именно GPU размещать тензор, передав отсчитываемый с нуля целочисленный номер GPU на машине, вот так:

In [56]:
points_gpu = points.to(device="cuda:0")

После этого все операции над тензором, например умножение всех элементов на константу, производится на GPU.

In [57]:
%%time

points = torch.tensor([[4.0, 1.0], [5.0, 3.0,], [2.0, 1.0]])
point = 2 * points

CPU times: total: 0 ns
Wall time: 1.01 ms


In [58]:
%%time

point_gpu = 2 * points.to(device='cuda')

CPU times: total: 1.69 s
Wall time: 1.69 s


**ОТМЕТИМ, что тензор points_gpu не передается обратно в CPU после вычисления результата. Вот что происходит:**

1. Тензор points копируется в GPU.
2. Выделяется память в GPU под новый тензор, в котором будет хранится результат умножения.
3. Возвращается обращение к этому GPU тензору.

Следовательно, если мы прибавим к результату константу:

In [59]:
points_gpu = points_gpu + 4

Операции сложения будет по-прежнему производится в GPU и никакой информации в CPU передаваться не будет (если мы не будем выводить полученный тензор на экран или обращаться к нему). Для переноса тензора обратно в CPU необходимо указать в методе to аргумент cpu, вот так:

In [60]:
points_cpu = points_gpu.to(device='cpu')

Можно также для получения тогоже результата воспользоваться сокращенными методами cpu и cuda вместо метода to:

In [61]:
points_gpu = points.cuda() # По умолчанию перекидывает на GPU с индексом 0
points_gpu = points.cuda(0) # Явно указываем, что перекидываем на нулевую GPU
points_cpu = points.cpu()

Стоит также упомянуть, с помощью метода to можно менять тип данных и их размещение одновременно, указав в качестве аргументов device и dtype.

***

## Совместимость с Numpy

Тензоры PyTorch можно очень эффективно преобразовывать в массивы NumPy и наоборот. Благодаря этому можно воспользоваться огромными объемами функциональности экосистемы Python, основанных на типах массивов NumPy.

Чтобы получить массив NumPy из нашего тензора points, достаточно вызвать:

In [62]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

Этот код возвращает многомерный массив NumPy нужного размера, формы и числового типа. Интересно то, что у возвращаемого массива один и тот же буфер с тензорным хранилищем. Это значит, что выполнение метода numpy практически не подразумевает накладных расходов, если данные хранятся на CPU, а также что модификация массива NumPy ведет к изменениям исходного тензора.

Если память под тензор выделяется в GPU, Pytorch копирует содержимое тензора в массив NumPy, расположенный в CPU.

И наоборот, вот так можно получить тензор PyTorch из массива NumPy:

In [63]:
points = torch.from_numpy(points_np)

С сохранением той же стратегии совместного использования буфера, которую мы только что описали.
***

## Обобщенные тензоры тоже тензоры

Pytorch вызывает правильные вычислительные ф-ции вне зависимости от того, размещен ли тензор в памяти CPU или GPU. Достигается это с помощью механизма **диспетчеризации** способного обслуживать другие типы тензоров посредством подключения API пользователя к нужным ф-циям прикладной части. Размуеется, существуют другие виды тензоров: некоторые специально предназначены для конкретных классов аппаратных устройств (например, TPU от Google), а стратегии представления данных у других массивов отличаются от плотных массивов, с которыми мы сталкивались до сих пор.

***

## Сериализация тензоров

Создавать тензоры по ходу дела удобно, но если внутри него ценные данные, желательно сохранить его в файл и загрузить потом обратно. В конце концов, мы же не хотим обучать модель с нуля каждый раз, когда запускаем программу! Для сериализации объектов-тензоров PyTorch использует "за кулисами" pickle, а также специализированный код сериализации для хранилища. Вот как можно сохранить наш тензор points в файл ourpoints.t:

In [64]:
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [65]:
torch.save(points, './data/ourpoints.t')

Либо можно передать дескриптор файла вместо его названия:

In [66]:
with open('./data/ourpoints.t', 'wb') as f:
    torch.save(points, f)

Загрузка тензора points обратно тоже выполняется одной строкой кода:

In [67]:
points = torch.load('./data/ourpoints.t')
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

Что эквивалентно:

In [68]:
del points

with open('./data/ourpoints.t', 'rb') as f:
    points = torch.load(f)
    
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

И хотя подобным образом можно быстро сохранять тензоры, если их нужно загружать только в PyTorch, сам по себе формат не отличается совместимостью: прочитать тензор с помощь какого-либо еще ПО, помимо PyTorch, **не получится**. В зависимости от сценария использования, это может и не ограничивать наши возможности, но в противном случае имеет смысл выяснить, как сохранять тензоры совместимым образом.

***

## Задания

Создать тензор из list(range(9))

In [69]:
res = torch.tensor(list(range(9)))
res.shape

torch.Size([9])

**view** - Возвращает новый тензор с теми же данными, что и собственный тензор, но другой формы.

In [70]:
a = torch.randn(9)
b = a.view(3, 3)
z = a.view(-1, 3)
print(a, '\n')
print(b, '\n')
print(z)

tensor([-0.4371, -0.5058, -0.6620, -1.0235,  2.9106, -0.7871,  0.3801,  1.2505,
        -0.5998]) 

tensor([[-0.4371, -0.5058, -0.6620],
        [-1.0235,  2.9106, -0.7871],
        [ 0.3801,  1.2505, -0.5998]]) 

tensor([[-0.4371, -0.5058, -0.6620],
        [-1.0235,  2.9106, -0.7871],
        [ 0.3801,  1.2505, -0.5998]])


In [71]:
c = b[1:,1:]
print(b, '\n')
print(c)

tensor([[-0.4371, -0.5058, -0.6620],
        [-1.0235,  2.9106, -0.7871],
        [ 0.3801,  1.2505, -0.5998]]) 

tensor([[ 2.9106, -0.7871],
        [ 1.2505, -0.5998]])


In [72]:
res = torch.cos(torch.tensor(list(range(9))))
res

tensor([ 1.0000,  0.5403, -0.4161, -0.9900, -0.6536,  0.2837,  0.9602,  0.7539,
        -0.1455])